### В ноутбуке: токенизирует описания к курсам, приводим в векторную форму, считаем меру близости и ранжируем 

#### spark.sesion

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("lab02")
         .getOrCreate())

sc = spark.sparkContext

#### Задача
По имеющимся данным портала eclass.cc построить content-based рекомендации по образовательным курсам. Запрещено использовать библиотеки pandas, sklearn и аналогичные.

#### Описание данных
Имеются следующие данные на вход:

- набор данных о всех курсах. Датасет можно взять с HDFS по адресу: `/labs/slaba02/DO_record_per_line.json`
- `id` курсов, для которых надо дать рекомендации (указаны в Личном кабинете)

Данные выглядят следующим образом:

```
{"lang": "en",
"name": "Accounting Cycle: The Foundation of Business Measurement and Reporting",
"cat": "3/business_management|6/economics_finance",
"provider": "Canvas Network",
"id": 4,
"desc": "This course introduces the basic financial statements used by most businesses, as well as the essential tools used to prepare them. This course will serve as a resource to help business students succeed in their upcoming university-level accounting classes, and as a refresher for upper division accounting students who are struggling to recall elementary concepts essential to more advanced accounting topics. Business owners will also benefit from this class by gaining essential skills necessary to organize and manage information pertinent to operating their business. At the conclusion of the class, students will understand the balance sheet, income statement, and cash flow statement. They will be able to differentiate between cash basis and accrual basis techniques, and know when each is appropriate. They\u2019ll also understand the accounting equation, how to journalize and post transactions, how to adjust and close accounts, and how to prepare key financial reports. All material for this class is written and delivered by the professor, and can be previewed here. Students must have access to a spreadsheet program to participate."}
```

In [3]:
!hdfs dfs -ls /labs/slaba02/DO_record_per_line.json

-rw-r--r--   3 hdfs hdfs   69519728 2022-01-06 18:46 /labs/slaba02/DO_record_per_line.json


In [4]:
courses_desc = spark.read.json('/labs/slaba02/DO_record_per_line.json')
courses_desc.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
+--------------------+--------------------+---+----+--------------------+--------------+
only showing top 5 rows



#### Результат
Для каждого id курса из личного кабинета необходимо дать топ-10 наиболее похожих на него курсов. Рекомендованные курсы должны быть того же языка, что и курс, для которого строится рекомендация.

Выходной формат — json — должен иметь следующую структуру:

```
{
  "123": [
    5372,
    16663,
    23114,
    13079,
    13084,
    ...
  ],
  "456": [
    ...
  ],
  "789": [
    ...
  ],
  "123456": [
    ...
  ],
  "456789": [
    ...
  ],
  "987654": [
    ...
  ]
}
```

#### Советы
Для подбора рекомендаций следует использовать меру TFIDF, а в качестве метрики для ранжирования — косинус угла между TFIDF-векторами для разных курсов.

Что такое TFIDF? TF — это term frequency: по сути, сколько раз слово встречается в этом документе. Если мы сделаем такой word count по каждому документу, то получим вектор, который как-то характеризует этот документ.

Если мы сравним вектора, рассчитав дистанцую между ними, то получим вывод – насколько похожи эти тексты. Назовем этот подход наивным.

Этот подход наивен, потому что мы как бы присваиваем одинаковый вес каждому слову, которое у нас есть в тексте. А что если мы попробуем как-то повысить значимость тех слов, которые часто встречаются только в этом тексте? Для этого мы посчитаем DF – document frequency: по сути, число документов, в которых есть вхождение этого слова. Мы хотим "штрафовать" слово за частое появление в документах, поэтому делаем инверсию этой величины – буква I в TFIDF. Теперь для каждого слова мы будем считать TF и делить на IDF. Так мы получим другой вектор для нашего документа. Он может быть более правильным для наших задач.

TFIDF нужно считать для описаний курсов (desc). При извлечении слов из описания словом считаем то, что состоит из латинских или кириллических букв или цифр, знаки препинания и прочие символы не учитываются.

Для поиска слов можно использовать такой код на Python (может быть проблема с распознаванием юникода).

```
regex = re.compile(u'[\w\d]{2,}', re.U)
regex.findall(string.lower())
```

Сам TFIDF реализован в Spark, писать с нуля вычисления не требуется. При вычислении TF с помощью HashingTF использовалось число фичей: 10000. То есть: `tf = HashingTF(10000)`.

#### tokenization

In [6]:
# Пайплайн = Препроцессинг + Токенизатор + HashingTF + IDF + join dataset'ов + cos_sim (udf) + формирование рек-ций

In [7]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF, Normalizer
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, col, lower, isnan, isnull, broadcast, desc, lower
from pyspark.sql.types import FloatType, ArrayType, StringType
import json
import re

In [8]:
courses_desc = spark.read.json('/labs/slaba02/DO_record_per_line.json')
courses_desc.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
+--------------------+--------------------+---+----+--------------------+--------------+
only showing top 5 rows



In [9]:
desc = courses_desc.select('id','lang','desc')

In [10]:
desc.count()

28153

In [11]:
desc.select('desc').distinct().count()

27810

In [12]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")

In [13]:
DATA = tokenizer.transform(desc)

In [14]:
stopwordListRus = StopWordsRemover.loadDefaultStopWords("russian")
stopwordListEng = StopWordsRemover.loadDefaultStopWords("english")
stopwordListEsp = StopWordsRemover.loadDefaultStopWords("spanish")


remover = StopWordsRemover(inputCol="words", 
                           outputCol="words_wo_stops" ,
                           stopWords=stopwordListRus + stopwordListEng + stopwordListEsp)

In [15]:
DATA = remover.transform(DATA)

In [16]:
DATA.show(3)

+---+----+--------------------+--------------------+--------------------+
| id|lang|                desc|               words|      words_wo_stops|
+---+----+--------------------+--------------------+--------------------+
|  4|  en|This course intro...|[this, course, in...|[course, introduc...|
|  5|  en|This online cours...|[this, online, co...|[online, course, ...|
|  6|  fr|This course is ta...|[this, course, is...|[course, taught, ...|
+---+----+--------------------+--------------------+--------------------+
only showing top 3 rows



#### compute TF

In [17]:
hasher_freq = HashingTF(numFeatures=10000, 
                        binary=True, 
                        inputCol='words_wo_stops', 
                        outputCol="word_vector_freq")
DATA_freq = hasher_freq.transform(DATA)

In [18]:
DATA_freq.show()

+---+----+--------------------+--------------------+--------------------+--------------------+
| id|lang|                desc|               words|      words_wo_stops|    word_vector_freq|
+---+----+--------------------+--------------------+--------------------+--------------------+
|  4|  en|This course intro...|[this, course, in...|[course, introduc...|(10000,[36,42,63,...|
|  5|  en|This online cours...|[this, online, co...|[online, course, ...|(10000,[32,222,29...|
|  6|  fr|This course is ta...|[this, course, is...|[course, taught, ...|(10000,[30,41,246...|
|  7|  en|We live in a digi...|[we, live, in, a,...|[live, digitally,...|(10000,[493,721,8...|
|  8|  en|This self-paced c...|[this, self-paced...|[self-paced, cour...|(10000,[32,65,115...|
|  9|  en|This game-based c...|[this, game-based...|[game-based, cour...|(10000,[56,268,30...|
| 10|  en|What’s in your di...|[what’s, in, your...|[what’s, digital,...|(10000,[1045,2044...|
| 11|  en|The goal of the D...|[the, goal, of, t..

In [19]:
DATA_freq.select('word_vector_freq').show(1,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|word_vector_freq                                                                                                                                                                                                                                                       

In [20]:
DATA_freq.count()

28153

#### compute idf

In [21]:
idf = IDF(inputCol="word_vector_freq", 
          outputCol="features").fit(DATA_freq)

In [22]:
tfidf = idf.transform(DATA_freq.select('id','lang','word_vector_freq'))

In [23]:
tfidf.take(1)#show(5, truncate=False)

[Row(id=4, lang='en', word_vector_freq=SparseVector(10000, {36: 1.0, 42: 1.0, 63: 1.0, 138: 1.0, 150: 1.0, 157: 1.0, 177: 1.0, 204: 1.0, 362: 1.0, 534: 1.0, 603: 1.0, 724: 1.0, 766: 1.0, 1023: 1.0, 1072: 1.0, 1355: 1.0, 1390: 1.0, 1446: 1.0, 1463: 1.0, 1523: 1.0, 1670: 1.0, 1697: 1.0, 1712: 1.0, 2015: 1.0, 2092: 1.0, 2414: 1.0, 2460: 1.0, 2523: 1.0, 2577: 1.0, 2757: 1.0, 3034: 1.0, 3231: 1.0, 3368: 1.0, 3496: 1.0, 3792: 1.0, 3834: 1.0, 3849: 1.0, 3869: 1.0, 3903: 1.0, 3986: 1.0, 4114: 1.0, 4140: 1.0, 4224: 1.0, 4295: 1.0, 4364: 1.0, 4372: 1.0, 4436: 1.0, 4978: 1.0, 5017: 1.0, 5374: 1.0, 6158: 1.0, 6245: 1.0, 6395: 1.0, 6470: 1.0, 6541: 1.0, 6642: 1.0, 6697: 1.0, 6863: 1.0, 7008: 1.0, 7282: 1.0, 7290: 1.0, 7298: 1.0, 7688: 1.0, 7735: 1.0, 7772: 1.0, 7779: 1.0, 7956: 1.0, 7973: 1.0, 8140: 1.0, 8164: 1.0, 8370: 1.0, 8534: 1.0, 8579: 1.0, 8624: 1.0, 8644: 1.0, 8922: 1.0, 9328: 1.0, 9347: 1.0, 9540: 1.0, 9605: 1.0, 9953: 1.0, 9970: 1.0}), features=SparseVector(10000, {36: 3.8519, 42: 4.4647

In [24]:
tfidf.count()

28153

In [25]:
tfidf.printSchema()

root
 |-- id: long (nullable = true)
 |-- lang: string (nullable = true)
 |-- word_vector_freq: vector (nullable = true)
 |-- features: vector (nullable = true)



In [26]:
tfidf = tfidf.select('id','lang','features')
tfidf.show()

+---+----+--------------------+
| id|lang|            features|
+---+----+--------------------+
|  4|  en|(10000,[36,42,63,...|
|  5|  en|(10000,[32,222,29...|
|  6|  fr|(10000,[30,41,246...|
|  7|  en|(10000,[493,721,8...|
|  8|  en|(10000,[32,65,115...|
|  9|  en|(10000,[56,268,30...|
| 10|  en|(10000,[1045,2044...|
| 11|  en|(10000,[87,157,15...|
| 12|  en|(10000,[161,164,8...|
| 13|  en|(10000,[26,1072,1...|
| 14|  en|(10000,[63,145,23...|
| 15|  en|(10000,[32,65,77,...|
| 16|  en|(10000,[32,273,30...|
| 17|  en|(10000,[695,2486,...|
| 18|  en|(10000,[316,364,6...|
| 19|  en|(10000,[768,855,1...|
| 20|  en|(10000,[273,317,4...|
| 21|  en|(10000,[148,157,1...|
| 22|  en|(10000,[128,177,2...|
| 23|  en|(10000,[332,527,6...|
+---+----+--------------------+
only showing top 20 rows



#### similarity calc for specific descriptions

In [27]:
## func 
cosine_similarity = udf(lambda v, u: float(v.dot(u) / (v.norm(2) * u.norm(2))), FloatType())

In [28]:
cosine_similarity = udf(lambda v, u: float(v.dot(u) / (v.norm(2) * u.norm(2))), FloatType())

# individual ids from personal account
film_list = [23126, 21617, 16627, 11556, 16704, 13702]
answers = {}

for id_ in film_list:
    single_data = (tfidf.filter(F.col('id') == id_)
                   .withColumnRenamed('id','single_id')
                   .withColumnRenamed('features','single_feature')
                   .join(
                       tfidf.withColumn('single_id', F.lit(id_)),
                       how='left',
                       on=['single_id','lang'])
                   .filter(F.col('id') != id_)
                   .withColumn('cosine', cosine_similarity(F.col('single_feature'), F.col('features')))
                   .filter(~F.isnan(F.col('cosine')))
                   .sort(F.col('cosine').desc())
                  )

    ids = single_data.select(F.col('id')).take(10)
    answers[id_] = [ids[i][0] for i in range(10)]
    
    print(str(id_) + ' is done')

23126 is done
21617 is done
16627 is done
11556 is done
16704 is done
13702 is done


In [29]:
answers

{23126: [25782, 23718, 7222, 14760, 23822, 11528, 24373, 5550, 13665, 25468],
 21617: [21609, 21608, 21616, 21492, 21700, 21716, 21703, 21706, 21587, 21618],
 16627: [11431, 12247, 16694, 5356, 9563, 5680, 5687, 23506, 17964, 23369],
 11556: [12679, 22710, 16488, 17910, 468, 387, 19394, 18005, 272, 12884],
 16704: [1236, 8186, 1164, 1365, 875, 8207, 8154, 1376, 1219, 20645],
 13702: [864, 21079, 13057, 1041, 1033, 915, 1217, 1216, 1173, 21025]}

In [32]:
with open("lab02.json", "w") as f:
    json.dump(answers, f)

In [30]:
spark.stop()